<a href="https://colab.research.google.com/github/Bhuto1998/Online-Cluster-Validity-Index/blob/main/Batch_Online_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm to Update Sample Covariance Matrix:
Let Sample covariance matrix be $S_{n} = \frac{1}{n-1} \sum_{i=1}^{n}(v_{i} - m_{n})(v_{i} - m_{n})^{T}$ where $v_{i}$'s are the datapoints and $m_{n}$ is the mean. Now let $m_{t}$ is the mean vector at time $t$. Our task is to determine the value of $S_{t+1}$ using $S_{t}$ and some key statistic. \\

 $S_{t+1} = \frac{1}{t} \sum_{i=1}^{t+1}(v_{i} - m_{t+1})(v_{i} - m_{t+1})^{T} = \frac{1}{t} \left[\sum_{i=1}^{t}(v_{i} - m_{t+1})(v_{i} - m_{t+1})^{T} + (v_{t+1} - m_{t+1})(v_{t+1}-m_{t+1})^{T} \right]$
 Let , $\delta_{t+1} = (v_{t+1} - m_{t+1})(v_{t+1}-m_{t+1})^{T}$ and $M_{t+1} = (m_{t}-m_{t+1})(m_{t}-m_{t+1})^{T}$ \\
 Now, $(v_{i}-m_{t+1})(v_{i}-m_{t+1})^{T} = (v_{i}-m_{t}+m_{t}-m_{t+1})(v_{i}-m_{t}+m_{t}-m_{t+1})^{T} \\= (v_{i}-m_{t})(v_{i}-m_{t})^{T} + (v_{i}-m_{t})(m_{t}-m_{t+1})^{T} + (m_{t}-m_{t+1})(v_{i}-m_{t})^{T} + (m_{t}-m_{t+1})(m_{t}-m_{t+1})^{T}$
 Notice that, $\sum_{i=1}^{t}(v_{i}-m_{t})(m_{t}-m_{t+1})^{T} = \sum_{i=1}^{t} (m_{t}-m_{t+1})(v_{i}-m_{t})^{T} = 0$ \\
 Hence, $S_{t+1} = \frac{1}{t}\left[ (t-1)S_{t} + tM_{t+1} + \delta_{t+1}  \right]$ 

 **Algorithm**
 1. Start from $S_{2} , m_{2}$
 2. For each t>2 and new data point $v_{t}$:
  * $M_{t} = (m_{t-1}-m_{t})(m_{t-1}-m_{t})^{T}$
  * $\delta_{t} = (v_{t} - m_{})(v_{t}-m_{t})^{T}$ 
  * $S_{t} = \frac{1}{t-1}\left[ (t-2)S_{t-1} + (t-1)M_{t} + \delta_{t}  \right
  ]$ 



# Batch-Online Clustering Algorithm Using Cluster Validity Index:

update(a,S) function takes a new point a and sample covariance S as input and returns the updated sample covariance. ocvi(a) function takes a new point as input and add it to the existing data points and returns the optimal cluster number. $f$ is a fixed value which is the number of samples required to call a cluster "stable".

1. For First $N$ points (typically 1000 or 2000): Find the $\mathcal{I}^{*}$ value. Let it be $K$.
2. Perform exponential fading SK-mean clustering with $\lambda = 0.9$ and evaluate the following:
 * centroids/mean points of each cluster: $C_{1},...,C_{K}$
 * counters (number of points): $n_{1},...,n_{K}$
 * Covariance matrices: $S_{1},....,S_{K}$
 * Maximum mohalanobis distance of two points in one cluster: $maxmoha = [m_{1},...,m_{K}]$

3. For each new point $x_{new}$ Perform the following: 
 * Find out the cluster which has the minimum mohalanobis distance from this point. Let it be cluster $r$ and let that distance be $d$.
 *  If $d < maxmoha[r]$:
    * $ p = unif(0,1)$
    * If $ p > \frac{d}{maxmoha[r]}$ or $ n_{r} < f: S_{r} = update(x_{new} , S_{r})$
  * else:
    * $t = ocvi(x_{new})$
    * If $t==K: S_{r} = update(x_{new},S_{r})$
    * Else: $C_{K+1} = x_{new}$ and $S_{K+1} = I$
  * $C_{r} = \frac{n_{r}C_{r} + x_{new}}{n_{r} + 1}$ and $n_{r} = n_{r} + 1$



# Reasoning:
In the first we wait to get a stable structure of the dataset. Note that we are using mohalanobis distance instead of regular euclidean distance to capture the deviation in terms of the distribution of the cluster and not just the cluster center. Now if the mohalanobis distance of the new point from the predicted cluster is smaller than the maximum mohalanobis distance that means it is highly likely the new point indeed belongs to the cluster and we have already seen points similar to that one. So we use it to update sample covariance probabilistically. Note that if the number of points observed for a cluster is less than a particular threshold ($f$) we say that the cluster is not yet stable (i.e. the sample covariance is not a good estimate of population covariance). Typically value of such $f$ depends on the dimension of the dataset. For univariate 35 points is good enough but for bivariate we might need more than 50 points. (This is based on large sample asymptotics of the sample covariance). In the case that the distance is larger than the maximum mohalanobis distance itself, we have a potential suspect for a new cluster itself. We run the ocvi function to check if the optimal number of cluster changed. If it did we add a new cluster and in the othercase we use that point to update the Sample covariance of the predicted cluster.  

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from sklearn.cluster import KMeans
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
#Function to update Covariance
def update(x_new,S,mean_old,n):
  temp = np.multiply(mean_old,n-1)
  temp = np.add(temp,x_new)
  mean_new = np.divide(temp,n) #evaluating updated mean

  #Next we will evaluate M_t
  temp = np.subtract(mean_old,mean_new)
  M_t = np.outer(temp,temp)

  #Evaluating delta_t
  temp = np.subtract(x_new,mean_new)
  delta_t = np.outer(temp,temp)

  #Evaluating S_t
  temp = np.multiply(M_t,n-1)
  temp = np.add(temp, delta_t)
  temp2 = np.multiply(S,n-2)
  S_t = np.add(temp2,temp)
  S_t = np.divide(S_t,n-1)

  return S_t , mean_new


In [ ]:
#Function to find mohalanobis distance of a point from a distribution
def mdist(x_new,S,m):
  t = len(x_new)
  temp = np.subtract(x_new,m)
  S = np.linalg.inv(S)
  temp2 = np.matmul(temp,S)
  print(temp2)
  temp3 = np.subtract(x_new,m)
  temp3.shape = (t,1)
  temp2 = np.matmul(temp2,temp3)
  print(temp2)
  temp2 = temp2 ** 0.5
  return temp2[0]


In [ ]:
#Some Useful Functions:
#Function-1: Finding the closest cluster of a point given the point and centroids
def find_closest_cluster(x, centroids):
    closest_cluster, closest_distance = 999999, 999999 # initially invalid
    K = len(centroids)
    for k in range(K): # no. clusters
        temp = 0
        for j in range(len(centroids[k])):
            temp = temp + (x[j]-centroids[k][j])**2
        distance = temp # Euclidean distance
        if distance < closest_distance:
            closest_cluster  = k
            closest_distance = distance
    return closest_cluster

#Function-2: Calculating Euclidean Norm
def norm(v):
    temp = 0
    for i in range(len(v)):
        temp = temp + (v[i]**2)
    temp = temp**0.5
    return temp


In [ ]:
#OCVI value when a new datapoint arrives
def E_K(x_new,l,p,g,centroids,counter):
  #Evaluating E_K and E_1 
  E_K = np.zeros(11)
  for r in range(11):
    K = r + 1
    closest_K = find_closest_cluster(x_new,centroids[r])
    m = counter[r].copy()
    counter[r][closest_K] +=1 
    temp3 = np.subtract(counter[r],m)
    temp_centroids = centroids[r].copy()
    temp = np.subtract(x_new,temp_centroids[closest_K])
    temp = np.multiply(temp, l)
    temp_centroids[closest_K] = np.add(temp_centroids[closest_K],temp)
    for i in range(K):
      temp1 = np.subtract(centroids[r][i],temp_centroids[i])
      temp1 = np.transpose(temp1)
      q = np.matmul(temp1,g[r][i])
      b = norm(np.subtract(centroids[r][i],temp_centroids[i]))**2
      a = temp3[i] * (norm(np.subtract(x_new,temp_centroids[i]))**2)
      p[r][i] = (l*p[r][i]) + (2*q*l) + (b*m[i]*l) + a
      g[r][i] = (l*g[r][i]) + np.multiply((l*m[i]),np.subtract(centroids[r][i],temp_centroids[i])) + np.multiply(temp3[i],np.subtract(x_new,temp_centroids[i]))
      centroids = temp_centroids.copy()
    E_K[r] = np.sum(p)
    return E_K , p , g, centroids , counter
def dmax(x_new,l,centroids,counter):
  d = []
  for r in range(1,10,1):
    K = r + 2
    closest_k = find_closest_cluster(x_n,centroids)
    counter[r][closest_k] +=1
    temp = np.subtract(x_new,centroids[r][closest_k])
    temp = np.multiply(temp,l)
    centroids[r][closest_k] = np.add(centroids[r][closest_k],temp)
    for h in range(len(centroids[r])):
      temp2 = 0
      for x in range(len(centroids[r])):
        v = np.linalg.norm(np.subtract(centroids[r][h],centroids[r][x]))
        if v>temp2:
         temp2 = v
    d = d + [temp2]
  return d , centroids , counter

def ocvi(E_K,dmax):
  o = []
  for i in range(1,10,1):
    t = np.divide(E_K[0],E_K[i])
    d = dmax2[i-1]
    t = np.multiply(t,np.multiply(d,d))
    t = t/((i+1)**2)
    o = o + [t]
  
  s = np.argmax(o)
  return s+1


In [ ]:
def eK2(K,X,l):
    if len(X)<K:
        print("length is an issue")
    else:
        centroids = X[:K]
        counter = np.ones(K)
        p = np.zeros(K)
        temp_len = len(X) - K
        eK = np.zeros(temp_len)
        temp_len = (K,len(X[K]))
        g = np.zeros(temp_len)
        
        for j in range(K,len(X),1):      
            closest_K = find_closest_cluster(X[j],centroids)
            m = counter.copy()
            counter[closest_K] +=1
            
            temp3 = np.subtract(counter,m)
            temp_centroids = centroids.copy()
            temp = np.subtract(X[j],temp_centroids[closest_K])
            temp = np.multiply(temp, l)
            temp_centroids[closest_K] = np.add(temp_centroids[closest_K],temp)
            #update stage
            for i in range(K):
                temp1 = np.subtract(centroids[i],temp_centroids[i])
                temp1 = np.transpose(temp1)
                q = np.matmul(temp1,g[i])
                b = norm(np.subtract(centroids[i],temp_centroids[i]))**2
                a = temp3[i] * (norm(np.subtract(X[j],temp_centroids[i]))**2)
                p[i] = (l*p[i]) + (2*q*l) + (b*m[i]*l) + a
                g[i] = (l*g[i]) + np.multiply((l*m[i]),np.subtract(centroids[i],temp_centroids[i])) + np.multiply(temp3[i],np.subtract(X[j],temp_centroids[i]))
            centroids = temp_centroids.copy()
            eK[j-K] = np.sum(p)
        return eK, p , g , counter
#Function-5: Finding d_max for exponentially fading setting
def dmax2(K,X,l):
    if len(X)<K:
        print("length is an issue")
    else:
        d = []
        centroids = X[:K]
        counter = np.ones(K)
        for x_n in X[K:]:
            closest_k = find_closest_cluster(x_n,centroids)
            counter[closest_k] +=1
            temp = np.subtract(x_n,centroids[closest_k])
            temp = np.multiply(temp,l)
            centroids[closest_k] = np.add(centroids[closest_k],temp)
            for h in range(len(centroids)):
                temp2 = 0
                for x in range(len(centroids)):
                    v = np.linalg.norm(np.subtract(centroids[h],centroids[x]))
                    if v>temp2:
                        temp2 = v
            d = d + [temp2]
        return d

        

In [ ]:
def batch_online(data,l):
  num_clust = np.zeros(len(data))
  #For first 2000 points we run batch clustering
  X = data[:2000]
  p = []
  g = []
  counter = [[],[],[],[],[],[],[],[],[],[],[]]
  I = [[],[],[],[],[],[],[],[],[],[]]
  eK1,p2,g2,counter2 = eK2(1,X,l)
  for K in range(2,12,1):
    d = dmax2(K,X,l)
    eK,p1,g1,counter1 = eK2(K,X,l)
    p = p + [p1]
    g = g+ [g1]
    counter[K-1] = counter[K-1] + [counter1]
    
    t = np.divide(eK1[(K-1):],eK)
    t = np.multiply(t,np.multiply(d,d))
    t = t/(K*K)
    I[K]= I[K]+[t]
  counter[0] = counter[0] + [counter2]




In [ ]:
x_new = [2,3,1]
S = np.eye(3)
mean_old = [1,2.3,4]
mdist(x_new,S,mean_old)

[ 1.   0.7 -3. ]
[10.49]


3.2388269481403293

In [ ]:
norm([2,3,1])

3.7416573867739413

In [ ]:
mean_new

array([1.002 , 2.3014, 3.994 ])

In [ ]:
a = np.array([3,2,1])
a.shape = (3,1)
a

array([[3],
       [2],
       [1]])

In [ ]:
a = np.array([3,2,1])

array([3, 2, 1])

In [ ]:
np.matmul([3,2,1],np.eye(3))

array([3., 2., 1.])

In [ ]:
p = [[1,2],[1,2,3,4],[1]]

In [5]:
t = [] 
r = [[1,2],[3,4]]
t = t + [r]
t = t + [0]
t + [[[2,1],[2,3]]]

[[[1, 2], [3, 4]], 0, [[2, 1], [2, 3]]]

In [7]:
I = [[],[],[],[],[],[],[],[],[],[]]

In [8]:
I[2] + [3]

[3]

In [9]:
I

[[], [], [], [], [], [], [], [], [], []]

In [11]:
I[2] = I[2]+ [3] + [5]
I

[[], [], [3, 3, 5], [], [], [], [], [], [], []]